# Reinforcement Learning (RL)

## Human-level control through deep rl [[Paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)]
## Playing Atari with Deep RL [[Paper](https://arxiv.org/abs/1312.5602)]


## Installs


In [ ]:
%pip install gym==0.26.1 --quiet
%pip install gym[atari] --quiet
%pip install autorom[accept-rom-license] --quiet

%pip install comet_ml --quiet
%pip install tensorboardX --quiet
%pip install onnx --quiet

## Comet Config

In [ ]:
import comet_ml
comet_ml.init()

## Imports


In [ ]:
import random
from collections import deque

import cv2
import numpy as np
import torch
import torch.nn as nn
import tensorboardX as tb

import gym
from gym import spaces, wrappers

try:
    from tqdm.auto import trange
except Exception:
    trange = range

print(f"gym=={gym.__version__}")
print(f"numpy=={np.__version__}")
print(f"torch=={torch.__version__}")
print(f"tensorboardX=={tb.__version__}")

## Wrappers

In [ ]:
cv2.ocl.setUseOpenCL(False)


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == "NOOP"

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(
                1, self.noop_max + 1
            )  # pylint: disable=E1101
        assert noops > 0
        obs, info = None, None
        for _ in range(noops):
            obs, _, done, _, info = self.env.step(self.noop_action)
            if done:
                obs, info = self.env.reset(**kwargs)
        return obs, info

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == "FIRE"
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _, info = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _, info = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs, info

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, truncated, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs, info = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _, info = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs, info


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros(
            (2, *env.observation_space.shape), dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, truncated, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, truncated, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work.
        Expects inputs to be of shape height x width x num_channels
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(self.height, self.width, 1), dtype=np.uint8
        )

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self.width, self.height), interpolation=cv2.INTER_AREA
        )
        return frame[:, :, None]


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._frames)

    def __getitem__(self, i):
        return self._frames[i]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        Expects inputs to be of shape num_channels x height x width.
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(shp[0] * k, shp[1], shp[2]), dtype=np.uint8
        )

    def reset(self):
        ob, info = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob(), info

    def step(self, action):
        ob, reward, done, trunc, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, trunc, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class PyTorchFrame(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env):
        super(PyTorchFrame, self).__init__(env)
        shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=(shape[-1], shape[0], shape[1]), dtype=np.uint8
        )

    def observation(self, observation):
        return np.rollaxis(observation, 2)


## Environment

In [ ]:
class TrainingEnvironment:
    def __init__(self, env_name, seed, steps):       
        assert "NoFrameskip" in env_name, "Require environment with no frameskip"

        self.seed = seed
        self.steps = steps

        self.env = gym.make(env_name, render_mode="rgb_array")
        self.env.seed(seed)

        # Investigate... rgb_array is not detected
        self.env = wrappers.RecordVideo(self.env, "videos", step_trigger=self.trigger)
        self.env = NoopResetEnv(self.env, noop_max=30)
        self.env = MaxAndSkipEnv(self.env, skip=4)
        self.env = EpisodicLifeEnv(self.env)
        self.env = FireResetEnv(self.env)
        self.env = ClipRewardEnv(self.env)
        self.env = WarpFrame(self.env)
        self.env = PyTorchFrame(self.env)
        self.env = FrameStack(self.env, k=4)

    def unwrap(self):
        return self.env
    
    def trigger(self, x):
        return x > self.steps - 600

    def __str__(self):
        return f"seed = {self.seed}\nstates = {self.env.observation_space.shape}\nactions = {self.env.action_space.n}"


## Replay Buffer


In [ ]:
class ReplayBuffer:
    """
    Simple storage for transitions from an environment.
    """

    def __init__(self,  N, observation_shape, num_actions):
        self.N = N
        self._mem_counter = 0
        self._observation_shape = observation_shape
        self._num_actions = num_actions

        self.states = np.zeros((self.N, *self._observation_shape), dtype=np.float32)
        self.actions = np.zeros(self.N, dtype=np.int64)
        self.rewards = np.zeros(self.N, dtype=np.float32)
        self.next_states = np.zeros((self.N, *self._observation_shape), dtype=np.float32)
        self.dones = np.zeros(self.N, dtype=np.float32)

    def __call__(self, indices):
        return self.states[indices],\
            self.actions[indices],\
            self.rewards[indices],\
            self.next_states[indices],\
            self.dones[indices]

    def add(self, state, action, reward, next_state, done):
        """
        Add a transition to the buffer. Old transitions will be overwritten if the buffer is full.
        :param state: the agent's initial state
        :param action: the action taken by the agent
        :param reward: the reward the agent received
        :param next_state: the subsequent state
        :param done: whether the episode terminated
        """
        next_index = self._mem_counter % self.N

        self.states[next_index] = np.array(state)
        self.actions[next_index] = action
        self.rewards[next_index] = reward
        self.next_states[next_index] = np.array(next_state)
        self.dones[next_index] = float(done)

        self._mem_counter += 1

    def sample(self, batch_size):
        """
        Randomly sample a batch of transitions from the buffer.
        :param batch_size: the number of transitions to sample
        :return: a mini-batch of sampled transitions
        """
        memory = min(self._mem_counter, self.N) - 1
        assert self._mem_counter >= batch_size, f"memory={memory} must have at least batch_size={batch_size} frames"

        indices = np.random.randint(0, memory, size=batch_size)

        return self(indices)

    def state_dict(self):
        dict = {
            'N': self.N,
            'mem_counter': self._mem_counter,
            'observation_shape': self._observation_shape,
            'num_actions': self._num_actions,
            'states': self.states,
            'actions': self.actions,
            'rewards': self.rewards,
            'next_state': self.next_states,
            'dones': self.dones,
        }

        return dict

    def load_state_dict(self, state_dict):
        self.N = state_dict['N']
        self._mem_counter = state_dict['mem_counter']
        self._observation_shape = state_dict['observation_shape']
        self._num_actions = state_dict['num_actions']

        self.states = np.array(state_dict['states'], dtype=np.float32)
        self.actions = np.array(state_dict['actions'], dtype=np.int64)
        self.rewards = np.array(state_dict['rewards'], dtype=np.float32)
        self.next_states = np.array(state_dict['next_states'], dtype=np.float32)
        self.dones = np.array(state_dict['dones'], dtype=np.float32)


## DQN Model

In [ ]:
class DQNModel(nn.Module):
    def __init__(self, num_input_channels, num_actions):
        super(DQNModel, self).__init__()

        # no. channels -> 32 features -> 64 features -> 64*7*7=3136 features -> 512 -> no. actions
        self.model = nn.Sequential(
            # First hidden layer
            nn.Conv2d(num_input_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            
            # Second hidden layer
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            
            # Third hidden layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            
            # Convolution to linear
            nn.Flatten(),
            
            # Final hidden layer
            nn.Linear(64*7*7, 512),
            nn.ReLU(),
            
            # Output layer
            nn.Linear(512, num_actions))

    def forward(self, input):
        return self.model(input)


## DQN Agent


In [ ]:
class DQNAgent:
    def __init__(self, env, memory_size, use_double_dqn, lr, batch_size, gamma, device, log_dir, comet_config, log_hists):
        self.observation_shape = env.observation_space.shape
        self.num_actions = env.action_space.n
        self.batch_size = batch_size
        self.gamma = gamma
        self.device = device
        self.use_double_dqn = use_double_dqn
        self.episode_rewards = [0.0]
        self.should_log_hists = log_hists

        # Tensorboard + Comet
        self.tb_w = tb.SummaryWriter(log_dir=log_dir, comet_config=comet_config)

        # Initialize replay memory D to capacity N
        self.D = ReplayBuffer(N=memory_size,
                              observation_shape=self.observation_shape,
                              num_actions=self.num_actions)

        # Initialize action-value function Q with random weights θ
        self.Q = DQNModel(self.observation_shape[0], self.num_actions).to(self.device)
        
        # Initialize target action-value function Q^ with weights θ_ = θ
        self.Q_ = DQNModel(self.observation_shape[0], self.num_actions).to(self.device)
        self.update_target_network()

        # Loss
        self.criterion = nn.MSELoss()

        # Adam optimizer for Q network
        self.optimizer = torch.optim.Adam(self.Q.parameters(), lr=lr)

        # Indexing
        self.num_optims = 0

    def act(self, state):
        """
        Select an action greedily from the Q-network given the state
        :param state: the current state
        :return: the action to take
        """
        # NB: Unsqueezed a layer for unit batch
        state = torch.tensor(np.array([state]), dtype=torch.float32, device=self.device) / 255

        with torch.no_grad():
            # select a_t = argmax a Q(s,a|θ)
            return self.Q(state).argmax(dim=1, keepdim=True).item()

    def optimise_td_loss(self):
        """
        Optimise the TD-error over a single minibatch of transitions
        :return: the loss
        """
        # Sample random minibatch of transitions (state, action, reward, next_state, done) from D
        batch = self.D.sample(self.batch_size)
        
        # Alternative to .gather()
        indices = np.arange(self.batch_size)
        
        next_states = torch.tensor(batch[3], device=self.device) / 255

        with torch.no_grad():  # Not used in gradient calculation
            if self.use_double_dqn:
                # a' = argmax a Q(s+1,a|θ)
                actions_ = self.Q.forward(next_states)\
                    .argmax(dim=1, keepdim=True)\
                    .squeeze()
                    

                # Q^(s+1,a'|θ_)
                max_values = self.Q_.forward(next_states)[indices, actions_]

            else:
                # max a' Q^(s+1,a'|θ_)
                # NB: torch.max() returns tuple(Tensor[values], Tensor[indices])
                max_values = self.Q_.forward(next_states)\
                    .max(dim=1, keepdim=True)[0]\
                    .squeeze()

            rewards = torch.tensor(batch[2], device=self.device)
            dones = torch.tensor(batch[4], device=self.device)
            
            # Set y_j
            # NB: * (1 - dones) handles episodes terminating at step j+1 (will be 0 and only rewards considered)
            Y = rewards + self.gamma * max_values * (1 - dones)

        states = torch.tensor(batch[0], device=self.device) / 255
        actions = torch.tensor(batch[1], device=self.device)
        
        # Q(s,a|θ)
        values = self.Q.forward(states)[indices, actions]

        # Perform a gradient descent step on (Y - Q(s,a|θ))^2 with respect to the network parameters θ
        loss = self.criterion(Y, values).to(self.device)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Log to tensorboard
        self.tb_w.add_scalar("data/loss", loss.item(), self.num_optims)

        if self.should_log_hists:
            # Expensive comp
            self.log_hists()

        self.num_optims += 1

    def update_target_network(self):
        """
        Update the target Q-network by copying the weights from the current Q-network
        """
        self.Q_.load_state_dict(self.Q.state_dict())

    def remember(self, state, action, reward, next_state, done):
        # Store transition (state, action, reward, next_state, done) in D
        self.D.add(state, action, reward, next_state, done)

        # Rewards
        self.episode_rewards[-1] += reward

        if done:
            self.episode_rewards.append(0.0)

    def num_episodes(self):
        return len(self.episode_rewards)    


    # Logging
    def log_hists(self):
        for name, weight in self.Q.named_parameters():
            self.tb_w.add_histogram('weights/Q-%s' % name, weight, self.num_optims)
            self.tb_w.add_histogram('weights/Q-%s.grad' % name, weight, self.num_optims)

        for name, weight in self.Q_.named_parameters():
            self.tb_w.add_histogram('weights/Q-target-%s' % name, weight, self.num_optims)
            self.tb_w.add_histogram('weights/Q-target-%s' % name, weight, self.num_optims)

    def log(self, t):
        self.tb_w.add_scalar("data/episodes", self.num_episodes(), t)

        mean_100ep_reward = round(np.mean(self.episode_rewards[-101:-1]), 1)
        self.tb_w.add_scalar("data/rewards", mean_100ep_reward, t)


    # Saving & Loading
    def save(self, path):
        print('saving checkpoint...')

        checkpoint = {
            'Q': self.Q.state_dict(),
            'Q_': self.Q_.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'loss': self.criterion,
            'D': self.D.state_dict()
        }

        torch.save(checkpoint, path)

    def load(self, path):
        print('loading checkpoints...')

        checkpoint = torch.load(path)

        self.Q.load_state_dict(checkpoint.get('Q'), map_location=self.device)
        self.Q.train()

        self.Q_.load_state_dict(checkpoint.get('Q_'),
                                map_location=self.device)

        self.optimizer.load_state_dict(checkpoint.get('optimizer'))

        self.criterion = checkpoint.get('loss')

        self.D.load_state_dict(checkpoint.get('D'))


## Args

In [ ]:
args_vars = {
    'device': "cuda" if torch.cuda.is_available() else "cpu",
    'log_hists': False,
    'log_dir': None,
    'stat_freq': 2,
    'out': 'data/model.pth',
    'checkpoint_freq': int(1e4),
    'comet_config': { 'disabled': False }
}

## Hyperparameters

In [ ]:
hyper_params = {
    "seed": 1,
    "env": "PongNoFrameskip-v4",
    "use-double-dqn": False,
    "replay-buffer-size": int(5e3),
    "batch-size": 32,
    "num-steps": int(1e6),
    "learning-starts": int(5e4),
    "learning-freq": 4,
    "target-update-freq": int(1e4),
    "learning-rate": int(1e-4),
    "discount-factor": 0.99,
    "eps-start": 1.0,
    "eps-end": 0.1,
    "eps-fraction": 0.1,
}

## Utils

In [ ]:
def log_onnx(agent, state):
        state = torch.tensor(np.array([state]), dtype=torch.float32, device=agent.device) / 255        
        filename = 'data/model.onnx'
        input_axis = { 0: "batch_size", 1: "channel", 2: "height", 3: "width"}
        output_axis = { 0: "action" }
        
        torch.onnx.export(agent.Q, 
                          (state,), 
                          filename,
                          export_params=True,
                          opset_version=10,
                          do_constant_folding=True,
                          input_names=["input"],
                          output_names=["output"],
                          dynamic_axes={ "input": input_axis, "output": output_axis })
        
        agent.tb_w.add_onnx_graph(filename)

def get_epsilon_threshold(t, hyper_params):
    eps_end = hyper_params['eps-end']
    eps_start = hyper_params['eps-start']
    eps_fraction = hyper_params['eps-fraction']
    steps = hyper_params['num-steps']
    
    fraction = min(1.0, float(t) / (eps_fraction * float(steps)))
    return eps_start + fraction * (eps_end - eps_start)

## Training Loop


In [ ]:
def train():
    # default_device = "cuda" if torch.cuda.is_available() else "cpu"

    # args_vars = setup_args(default_device=default_device)
    # hyper_params = args_to_hyper_params(args_vars)
    
    device = torch.device(args_vars.get("device"))
    print("device = %s" % device)

    np.random.seed(hyper_params['seed'])
    random.seed(hyper_params['seed'])

    train_env = TrainingEnvironment(env_name=hyper_params['env'],
                                    seed=hyper_params['seed'],
                                    steps=hyper_params['num-steps'])

    env = train_env.unwrap()
    

    agent = DQNAgent(env=env, 
                     memory_size=hyper_params['replay-buffer-size'], 
                     use_double_dqn=hyper_params['use-double-dqn'],
                     lr=hyper_params['learning-rate'],
                     batch_size=hyper_params['batch-size'],
                     gamma=hyper_params['discount-factor'],
                     device=device,
                     log_dir=args_vars['log_dir'],
                     comet_config=args_vars['comet_config'],
                     log_hists=args_vars['log_hists'])


    in_file = args_vars.get('in')
    if in_file is not None:
        try:
            agent.load(in_file)
        except FileNotFoundError:
            print("model file not found: %s" % in_file)
        except Exception:
            pass


    state, _ = env.reset()
    agent.tb_w.add_hparams(hparam_dict=hyper_params, metric_dict={})
    log_onnx(agent, state)
    
    out_file = args_vars.get('out')
    checkpoint_freq = args_vars['checkpoint_freq']
    try:
        for t in trange(hyper_params['num-steps']):
            epsilon_threshold = get_epsilon_threshold(t, hyper_params)

            #  select random action if sample is less equal than eps_threshold
            if (t <= hyper_params['learning-starts'] or random.random() <= epsilon_threshold):
                action = env.action_space.sample()
            else:
                action = agent.act(state)

            # take step in env
            next_state, reward, done, _, _ = env.step(action)

            # add state, action, reward, next_state, float(done) to reply memory - cast done to float
            agent.remember(state=state,
                            action=action,
                            reward=reward,
                            next_state=next_state,
                            done=float(done))

            # update state
            state = next_state

            if done:
                state, _ = env.reset()
                
                if (agent.num_episodes() % args_vars.get('stat_freq') == 0):
                    explore_time = int(100 * epsilon_threshold)
                    agent.log(t)
                    agent.tb_w.add_scalar("data/explore_time", explore_time, t)

            if t > hyper_params['learning-starts']:
                if t % hyper_params['learning-freq'] == 0:
                    agent.optimise_td_loss()

                # every C steps reset Q^ = Q
                if t % hyper_params['target-update-freq'] == 0:
                    agent.update_target_network()
                    
            if (out_file is not None and checkpoint_freq is not None and t % checkpoint_freq == 0):
                agent.save(out_file)
                    
        print("total episodes: ", agent.num_episodes())

    finally:
        agent.tb_w.close()
        if out_file is not None:
            agent.save(out_file)


## Main

In [ ]:
train()